<a href="https://colab.research.google.com/github/lkalilduarte/Desafios/blob/main/teste_deploy_desafio_fieldpro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importação de bibliotecas e do dataframe
#!pip install pycaret
#from pycaret import *
import pandas as pd
import numpy as np
import sklearn.preprocessing
from sklearn.model_selection import train_test_split #permite separacao das bases de dados de amostragem aleatoria
from sklearn.linear_model import LinearRegression # traz a regressao linear
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm

data_sensor_read = pd.read_csv('https://raw.githubusercontent.com/lkalilduarte/Desafios/main/Estacao_Convencional.csv')
data_estacao_convencional_read = pd.read_csv('https://raw.githubusercontent.com/lkalilduarte/Desafios/main/Sensor_FieldPRO.csv')

In [ ]:
#Visualizando os dados do Sensor FieldPRO
data_sensor_read.head(50)

,data,Hora (Brasília),chuva
0,2020-09-01,00:00:00,0.0
1,2020-09-01,01:00:00,0.0
2,2020-09-01,02:00:00,0.0
3,2020-09-01,03:00:00,0.0
4,2020-09-01,04:00:00,0.0
5,2020-09-01,05:00:00,0.0
6,2020-09-01,06:00:00,0.0
7,2020-09-01,07:00:00,0.0
8,2020-09-01,08:00:00,0.0
9,2020-09-01,09:00:00,0.0


In [ ]:
#Identificando o tipo dos dados nas colunas dataframe Sensor FieldPRO
data_sensor_read.dtypes

Datetime – utc          object
air_humidity_100       float64
air_temperature_100    float64
atm_pressure_main        int64
num_of_resets            int64
piezo_charge             int64
piezo_temperature        int64
dtype: object

In [ ]:
#Verificando se existem valores nan e contá-los do dataframe Sensor FieldPRO
valores_nan_sensor_fieldpro = []

for coluna in data_sensor_read.columns:

  contagem_nan_sensor_fieldpro = data_sensor_read.loc[pd.isna(data_sensor_read[coluna]), coluna].shape[0]

  valores_nan_sensor_fieldpro.append(contagem_nan_sensor_fieldpro)

valores_nan_sensor_fieldpro

[0, 6, 6, 0, 0, 0, 0]

In [ ]:
#Verificando se existem valores null do dataframe Sensor FieldPRO
valores_null_sensor_fieldpro = []

for coluna in data_sensor_read.columns:

  contagem_null_sensor_fieldpro = data_sensor_read.loc[pd.isnull(data_sensor_read[coluna]), coluna].shape[0]

  valores_null_sensor_fieldpro.append(contagem_null_sensor_fieldpro)

valores_null_sensor_fieldpro

[0, 6, 6, 0, 0, 0, 0]

In [ ]:
#O resultado das duas verificações foi o mesmo, indicando que os valores que estão sendo
# identificados como NaN são os mesmos que estão sendo identificados comp Null
# portanto só será necessário um tratamento para esses valores

In [ ]:
pd.isnull(data_sensor_read['air_temperature_100'][1])

True

In [ ]:
#Foram identificados 6 valores NaN, tanto na coluna air_humidity_100 quanto na air_temperature_100
#Agora será feito um tratamento para substituir os valores NaN pela média dos valores de temperatura do ar ou humidade do ar nos
# horários adjacentes, por exemplo no horário 2020-10-01T00:00:00Z	os valores de temperatura do ar e humadidade do ar são NaN
# eles serão substituidos pela média dos valores de temperatura do ar e humidade do ar dos horários 2020-09-30T23:00:00Z
# e 2020-10-01T01:00:00Z, que são o horário antes e o depois da ocorrência do valor NaN
#O objetivo é substituir os valores NaN pois eles interferem nos cáculos e modelos que serão desenvolvidos mais a abaixo.

In [ ]:
#Criando uma lista com os valores da coluna de humidade do ar para poder substituir os valores NaN
lista_air_humidity_original = list(data_sensor_read['air_humidity_100'])

In [ ]:
#O for é usado para verificar quais são os valores NaN e substuir pela média dos valores anterior e posterior
#Para a coluna de humidade do ar
lista_air_humidity_semNaN = []
for i in range(len(lista_air_humidity_original)):
  if pd.isna(lista_air_humidity_original[i]) == True :
    lista_air_humidity_original[i] = (lista_air_humidity_original[i-1] + lista_air_humidity_original[i+1])/2

  lista_air_humidity_semNaN.append(lista_air_humidity_original[i])

lista_air_humidity_semNaN

[38.0,
 38.5,
 39.0,
 39.0,
 38.0,
 39.0,
 39.0,
 39.0,
 40.0,
 41.0,
 41.0,
 44.0,
 44.0,
 44.0,
 44.0,
 44.0,
 37.0,
 30.0,
 20.0,
 22.0,
 27.0,
 32.0,
 34.0,
 39.0,
 39.0,
 57.0,
 62.0,
 66.0,
 61.0,
 70.0,
 69.5,
 69.0,
 62.0,
 39.0,
 28.0,
 23.0,
 19.0,
 17.0,
 16.0,
 16.0,
 18.0,
 18.0,
 23.0,
 29.0,
 31.0,
 33.0,
 44.0,
 53.0,
 46.0,
 62.0,
 59.0,
 67.0,
 69.0,
 58.0,
 67.0,
 64.0,
 47.0,
 29.0,
 25.0,
 22.0,
 21.0,
 17.0,
 14.0,
 13.0,
 15.0,
 17.0,
 23.0,
 31.0,
 31.0,
 58.0,
 59.0,
 64.0,
 66.0,
 70.0,
 72.0,
 74.0,
 75.0,
 77.0,
 77.0,
 72.0,
 64.0,
 54.0,
 49.0,
 43.0,
 38.0,
 36.0,
 34.0,
 33.0,
 33.0,
 35.0,
 43.0,
 53.0,
 60.0,
 63.0,
 66.0,
 69.0,
 72.0,
 74.0,
 75.0,
 77.0,
 77.0,
 80.0,
 78.0,
 76.0,
 60.0,
 54.0,
 46.0,
 38.0,
 34.0,
 30.0,
 29.0,
 26.0,
 28.0,
 30.0,
 39.0,
 47.0,
 54.0,
 61.0,
 65.0,
 69.0,
 68.0,
 68.0,
 72.0,
 78.0,
 83.0,
 85.0,
 83.0,
 80.0,
 61.0,
 43.0,
 36.0,
 30.0,
 27.0,
 21.0,
 20.0,
 19.0,
 20.0,
 19.0,
 28.0,
 32.0,
 43.0,
 50.0,
 58.0,

In [ ]:
#Criando uma lista com os valores da coluna de temperatura do ar para poder substituir os valores NaN
lista_air_temperature_original = list(data_sensor_read['air_temperature_100'])

In [ ]:
#O for é usado para verificar quais são os valores NaN e substuir pela média dos valores anterior e posterior
#Para a coluna de temperatura do ar
lista_air_temperature_semNaN = []
for i in range(len(lista_air_temperature_original)):
  if pd.isna(lista_air_temperature_original[i]) == True :
    lista_air_temperature_original[i] = (lista_air_temperature_original[i-1] + lista_air_temperature_original[i+1])/2

  lista_air_temperature_semNaN.append(lista_air_temperature_original[i])

lista_air_temperature_semNaN

[31.366,
 31.366,
 31.366,
 31.322,
 31.24,
 30.828,
 30.756999999999998,
 30.686,
 30.63,
 30.588,
 30.474,
 30.332,
 30.403,
 30.474,
 30.516,
 30.346,
 32.424,
 34.502,
 39.018,
 37.91,
 34.397,
 30.884,
 30.276,
 28.49,
 27.98,
 22.626,
 22.922,
 20.67,
 22.116,
 20.358,
 20.11,
 19.862,
 22.356,
 30.108,
 34.518,
 37.684,
 40.396,
 42.156,
 42.52,
 41.984,
 39.868,
 39.968,
 35.74,
 31.566,
 30.134,
 29.48,
 25.386,
 23.774,
 25.132,
 21.872,
 22.172,
 21.066,
 20.302,
 22.612,
 20.512,
 22.368,
 28.634,
 33.51,
 37.37,
 40.452,
 40.664,
 41.7,
 42.506,
 42.65,
 40.848,
 38.804,
 35.312,
 30.176,
 29.766,
 27.88,
 26.024,
 23.59,
 21.928,
 20.868,
 20.174,
 19.62,
 19.168,
 18.502,
 18.292,
 19.452,
 21.56,
 24.79,
 26.804,
 29.438,
 32.29,
 33.282,
 34.786,
 35.354,
 35.51,
 34.7,
 31.408,
 28.222,
 26.38,
 24.89,
 23.688,
 22.582,
 21.532,
 20.658,
 20.048,
 19.508,
 19.24,
 18.732,
 18.788,
 19.976,
 23.912,
 26.224,
 29.822,
 33.99,
 35.838,
 37.71,
 38.094,
 39.728,
 37.286,


In [ ]:
#Inserindo as listas sem valores NaN no lugar das colunas originais
data_sensor_read['air_humidity_100'] = lista_air_humidity_semNaN
data_sensor_read['air_temperature_100'] = lista_air_temperature_semNaN

data_sensor_read

,Datetime – utc,air_humidity_100,air_temperature_100,atm_pressure_main,num_of_resets,piezo_charge,piezo_temperature
0,2020-09-30T23:00:00Z,38.0,31.366,9412,0,45123,30
1,2020-10-01T00:00:00Z,38.5,31.366,9412,0,45025,31
2,2020-10-01T01:00:00Z,39.0,31.366,9419,0,44923,31
3,2020-10-01T02:00:00Z,39.0,31.322,9419,0,44825,31
4,2020-10-01T03:00:00Z,38.0,31.240,9416,0,44728,31
...,...,...,...,...,...,...,...
1706,2020-12-11T09:00:00Z,82.0,19.480,9422,3,20669,17
1707,2020-12-11T10:00:00Z,69.0,24.000,9429,3,20606,19
1708,2020-12-11T11:00:00Z,63.0,25.440,9436,3,20529,25
1709,2020-12-11T12:00:00Z,60.0,27.160,9439,3,20442,28


In [ ]:
#Identifiquei visualmente que um dos horários está fora do padrão de 00 minutos e 00 segundos
# como irei usar a coluna Datetime – utc	como a coluna index os valores precisam estar no padrão
#Os caracteres de minuto e segundo do horário ficam nas posições de 14 a 19, então esses serão os
# caracteres verificados para padronizar

In [ ]:
#Criando uma lista com os valores da coluna de datetime para poder padronizar os valores de horário
lista_datetime_original = list(data_sensor_read['Datetime – utc'])

In [ ]:
#O for é usado para verificar quais são os valores com caracteres de minuto e segundo fora do padrão 00:00
# e substuir por 00:00

lista_datetime_padrao = []
for i in range(len(lista_datetime_original)):
  str(lista_datetime_original[i])
  if (lista_datetime_original[i][14:19]) != '00:00':
    lista_datetime_original[i] = lista_datetime_original[i][:14] + '00:00Z'

  lista_datetime_padrao.append(lista_datetime_original[i])

lista_datetime_padrao

['2020-09-30T23:00:00Z',
 '2020-10-01T00:00:00Z',
 '2020-10-01T01:00:00Z',
 '2020-10-01T02:00:00Z',
 '2020-10-01T03:00:00Z',
 '2020-10-01T04:00:00Z',
 '2020-10-01T05:00:00Z',
 '2020-10-01T06:00:00Z',
 '2020-10-01T07:00:00Z',
 '2020-10-01T08:00:00Z',
 '2020-10-01T09:00:00Z',
 '2020-10-01T11:00:00Z',
 '2020-10-01T12:00:00Z',
 '2020-10-01T13:00:00Z',
 '2020-10-01T14:00:00Z',
 '2020-10-01T15:00:00Z',
 '2020-10-01T16:00:00Z',
 '2020-10-01T18:00:00Z',
 '2020-10-01T19:00:00Z',
 '2020-10-01T20:00:00Z',
 '2020-10-01T21:00:00Z',
 '2020-10-01T22:00:00Z',
 '2020-10-01T23:00:00Z',
 '2020-10-02T00:00:00Z',
 '2020-10-02T01:00:00Z',
 '2020-10-02T03:00:00Z',
 '2020-10-02T04:00:00Z',
 '2020-10-02T05:00:00Z',
 '2020-10-02T06:00:00Z',
 '2020-10-02T07:00:00Z',
 '2020-10-02T08:00:00Z',
 '2020-10-02T09:00:00Z',
 '2020-10-02T10:00:00Z',
 '2020-10-02T11:00:00Z',
 '2020-10-02T12:00:00Z',
 '2020-10-02T14:00:00Z',
 '2020-10-02T15:00:00Z',
 '2020-10-02T16:00:00Z',
 '2020-10-02T17:00:00Z',
 '2020-10-02T18:00:00Z',


In [ ]:
#Inserindo a lista com datetime padronizado no dataframe Sensor FieldPRO
data_sensor_read['Datetime – utc'] = lista_datetime_padrao

data_sensor_read.head(50)

,Datetime – utc,air_humidity_100,air_temperature_100,atm_pressure_main,num_of_resets,piezo_charge,piezo_temperature
0,2020-09-30T23:00:00Z,38.0,31.366,9412,0,45123,30
1,2020-10-01T00:00:00Z,38.5,31.366,9412,0,45025,31
2,2020-10-01T01:00:00Z,39.0,31.366,9419,0,44923,31
3,2020-10-01T02:00:00Z,39.0,31.322,9419,0,44825,31
4,2020-10-01T03:00:00Z,38.0,31.240,9416,0,44728,31
5,2020-10-01T04:00:00Z,39.0,30.828,9411,0,44632,30
6,2020-10-01T05:00:00Z,39.0,30.757,9411,0,44537,30
7,2020-10-01T06:00:00Z,39.0,30.686,9413,0,44441,30
8,2020-10-01T07:00:00Z,40.0,30.630,9415,0,44347,30
9,2020-10-01T08:00:00Z,41.0,30.588,9419,0,44251,30


In [ ]:
#Entre as colunas numéricas do dataframe apenas as relativas a temperatura
# poderiam incluir valores negativos, ou seja, as colunas air_temperature_100
# e piezo_temperature podem ter valores negativos
#A humidade do ar, a pressão atmosférica, numéro de resets e a carga acumulada
# não poderiam ter valores negativos, portanto as colunas air_humidity_100,
#	atm_pressure_main, num_of_resets e piezo_charge devem ter apenas valores positivos

In [ ]:
#Criando um dataframe apenas com as colunas que não podem ter valores negativos
data_sensor_numerics = data_sensor_read[['air_humidity_100',	'atm_pressure_main',	'num_of_resets',	'piezo_charge']]

In [ ]:
#Verificando se existem valores negativos nas colunas
valores_negativos_sensor_fieldpro = []
for coluna in data_sensor_numerics:
  contagem_negativos_sensor_fieldpro = len(data_sensor_numerics.loc[data_sensor_numerics[coluna] < 0])
  valores_negativos_sensor_fieldpro.append(contagem_negativos_sensor_fieldpro)

valores_negativos_sensor_fieldpro

[0, 0, 0, 0]

In [ ]:
#Concluisse que não há valores negativos

In [ ]:
#Ajustando o nome da coluna Datetime - utc do dataframe Sensor FiledPRO pois é
# melhor que o nome da coluna não tenha espaços entre as palavras
data_sensor_read = data_sensor_read.rename(columns={"Datetime – utc": "Datetime"})

In [ ]:
#Criando um novo dataframe Sensor FieldPRO que indica que é a versão pronta para análises
data_sensor_pronto = data_sensor_read

In [ ]:
data_sensor_pronto.head(50)

,Datetime,air_humidity_100,air_temperature_100,atm_pressure_main,num_of_resets,piezo_charge,piezo_temperature
0,2020-09-30T23:00:00Z,38.0,31.366,9412,0,45123,30
1,2020-10-01T00:00:00Z,38.5,31.366,9412,0,45025,31
2,2020-10-01T01:00:00Z,39.0,31.366,9419,0,44923,31
3,2020-10-01T02:00:00Z,39.0,31.322,9419,0,44825,31
4,2020-10-01T03:00:00Z,38.0,31.240,9416,0,44728,31
5,2020-10-01T04:00:00Z,39.0,30.828,9411,0,44632,30
6,2020-10-01T05:00:00Z,39.0,30.757,9411,0,44537,30
7,2020-10-01T06:00:00Z,39.0,30.686,9413,0,44441,30
8,2020-10-01T07:00:00Z,40.0,30.630,9415,0,44347,30
9,2020-10-01T08:00:00Z,41.0,30.588,9419,0,44251,30


In [ ]:
#Fim da análise dos dados e ajuste do dataframe Sensor FieldPRO

In [ ]:
#Início da análise do dataframe da Estação Convencional

In [ ]:
#Visualizando os dados do dataframe da Estação Convencional
data_estacao_convencional_read.head(50)

,data,Hora (Brasília),chuva
0,2020-09-01,00:00:00,0.0
1,2020-09-01,01:00:00,0.0
2,2020-09-01,02:00:00,0.0
3,2020-09-01,03:00:00,0.0
4,2020-09-01,04:00:00,0.0
5,2020-09-01,05:00:00,0.0
6,2020-09-01,06:00:00,0.0
7,2020-09-01,07:00:00,0.0
8,2020-09-01,08:00:00,0.0
9,2020-09-01,09:00:00,0.0


In [ ]:
#Identificando o tipo dos dados nas colunas do dataframe Estação Convencional
data_estacao_convencional_read.dtypes

data                object
Hora (Brasília)     object
chuva              float64
dtype: object

In [ ]:
#Verificando se existem valores nan e contá-los do dataframe Estação Convencional
valores_nan_estacao_convencional = []

for coluna in data_estacao_convencional_read.columns:

  contagem_nan_estacao_convencional = data_estacao_convencional_read.loc[pd.isna(data_estacao_convencional_read[coluna]), coluna].shape[0]

  valores_nan_estacao_convencional.append(contagem_nan_estacao_convencional)

valores_nan_estacao_convencional

[0, 0, 0]

In [ ]:
#Não há valores NaN nesse dataframe
#Agora farei a verificação de há valores negativos de chuva, pois não deveriam existir

In [ ]:
#Criando um dataframe apenas com a coluna de chuva
data_estacao_convencional_numerics = data_estacao_convencional_read[['chuva']]

In [ ]:
#Verificando se existem valores negativos na coluna chuva
valores_negativos_estacao_convencional = []
for coluna in data_estacao_convencional_numerics:
  contagem_negativos_estacao_convencional = len(data_estacao_convencional_numerics.loc[data_estacao_convencional_numerics[coluna] < 0])
  valores_negativos_estacao_convencional.append(contagem_negativos_estacao_convencional)

valores_negativos_estacao_convencional

[0]

In [ ]:
#Não há valores negativos na coluna chuva
#Agora será feito uma concatenação e ajuste entre as colunas data e Hora (Brasília)
# do dataframe Estação Convencional para criar uma coluna no mesmo padrão que a
# coluna Datetime – utc	do dataframe Sensor FieldPRO, pois irei usar essas duas
# colunas para relacionar os dois dataframes

In [ ]:
#As listas com os valores das coluas de data e Hora (Brasília) do dataframe Estação
# Convencional foram criadas para execurtar o FOR abaixo
lista_data_estacao_convencional = data_estacao_convencional_read['data']
lista_horario_estacao_convencional = data_estacao_convencional_read['Hora (Brasília)']

In [ ]:
#O for é usado para juntar os valores das colunas data e Hora (Brasília) do dataframe Estação Convencional
# e deixar no no mesmo formato que a coluna Datetimae - utc do dataframe Sensor FielPRO

lista_datatime_estacao_convencional = []

for i in range(len(lista_data_estacao_convencional)):
  datetime_estacao_convencional = lista_data_estacao_convencional[i] + 'T' + lista_horario_estacao_convencional[i] + 'Z'

  lista_datatime_estacao_convencional.append(datetime_estacao_convencional)

lista_datatime_estacao_convencional

['2020-09-01T00:00:00Z',
 '2020-09-01T01:00:00Z',
 '2020-09-01T02:00:00Z',
 '2020-09-01T03:00:00Z',
 '2020-09-01T04:00:00Z',
 '2020-09-01T05:00:00Z',
 '2020-09-01T06:00:00Z',
 '2020-09-01T07:00:00Z',
 '2020-09-01T08:00:00Z',
 '2020-09-01T09:00:00Z',
 '2020-09-01T10:00:00Z',
 '2020-09-01T11:00:00Z',
 '2020-09-01T12:00:00Z',
 '2020-09-01T13:00:00Z',
 '2020-09-01T14:00:00Z',
 '2020-09-01T15:00:00Z',
 '2020-09-01T16:00:00Z',
 '2020-09-01T17:00:00Z',
 '2020-09-01T18:00:00Z',
 '2020-09-01T19:00:00Z',
 '2020-09-01T20:00:00Z',
 '2020-09-01T21:00:00Z',
 '2020-09-01T22:00:00Z',
 '2020-09-01T23:00:00Z',
 '2020-09-02T00:00:00Z',
 '2020-09-02T01:00:00Z',
 '2020-09-02T02:00:00Z',
 '2020-09-02T03:00:00Z',
 '2020-09-02T04:00:00Z',
 '2020-09-02T05:00:00Z',
 '2020-09-02T06:00:00Z',
 '2020-09-02T07:00:00Z',
 '2020-09-02T08:00:00Z',
 '2020-09-02T09:00:00Z',
 '2020-09-02T10:00:00Z',
 '2020-09-02T11:00:00Z',
 '2020-09-02T12:00:00Z',
 '2020-09-02T13:00:00Z',
 '2020-09-02T14:00:00Z',
 '2020-09-02T15:00:00Z',


In [ ]:
#Inserindo a coluna Datetime no dataframe Estação Convencional
data_estacao_convencional_read['Datetime'] = lista_datatime_estacao_convencional
data_estacao_convencional_read

,data,Hora (Brasília),chuva,Datetime
0,2020-09-01,00:00:00,0.0,2020-09-01T00:00:00Z
1,2020-09-01,01:00:00,0.0,2020-09-01T01:00:00Z
2,2020-09-01,02:00:00,0.0,2020-09-01T02:00:00Z
3,2020-09-01,03:00:00,0.0,2020-09-01T03:00:00Z
4,2020-09-01,04:00:00,0.0,2020-09-01T04:00:00Z
...,...,...,...,...
2251,2020-12-03,19:00:00,0.0,2020-12-03T19:00:00Z
2252,2020-12-03,20:00:00,0.0,2020-12-03T20:00:00Z
2253,2020-12-03,21:00:00,0.0,2020-12-03T21:00:00Z
2254,2020-12-03,22:00:00,0.0,2020-12-03T22:00:00Z


In [ ]:
#Criando um novo dataframe Estação Convencional apenas com as colunas Datetime e chuva
# pois apenas elas serao usadas daqui para frente
#E verificando o tipo dos dados das colunas para garantir que estão corretos
data_estacao_convencional_pronto = data_estacao_convencional_read[['Datetime', 'chuva']]
data_estacao_convencional_pronto.dtypes

Datetime     object
chuva       float64
dtype: object

In [ ]:
#Os tipos dos dados estão corretos
#Fim da análise dos dados e ajuste do dataframe Sensor FieldPRO

In [ ]:
#Início da análise comparativa entre os dois dataframes para poder usar os dados de chuva
# do dataframe Estação Convencional junto com os dados do dataframe Sensor FieldPRO
# para criar um modelo preditivo

In [ ]:
#Verificando as datas e horários que iniciam e terminam os dois dataframes para
# identificar a intersecção dos dados, pois essa intersecção que será usada no modelo
inicio_sensor_fielpro = data_sensor_pronto['Datetime'].min()
fim_sensor_fielpro = data_sensor_pronto['Datetime'].max()
inicio_estacao_convencional = data_estacao_convencional_pronto['Datetime'].min()
fim_estacao_convencional = data_estacao_convencional_pronto['Datetime'].max()

In [ ]:
#Criando uma tabela para comparar os resultados de Acurácia, precision e recall dos 4 cenários
datas_inicio_fim = [['Sensor_FieldPRO', inicio_sensor_fielpro, fim_sensor_fielpro], ['Estacao_Convecional', inicio_estacao_convencional, fim_estacao_convencional]]
comparacao_datas = pd.DataFrame(datas_inicio_fim, columns=['Dataframe ', 'Data_inicio', 'Data_fim'])
print(comparacao_datas)

            Dataframe            Data_inicio              Data_fim
0      Sensor_FieldPRO  2020-09-30T23:00:00Z  2020-12-11T13:00:00Z
1  Estacao_Convecional  2020-09-01T00:00:00Z  2020-12-03T23:00:00Z


In [ ]:
#Criando as variáveis data_inicio e data_fim que são os datetimes de início e fim
# da instersecção dos dois dataframes
data_inicio = inicio_sensor_fielpro
data_fim = fim_estacao_convencional

In [ ]:
#Com essa comparação mostra que o dataframe Estação Convencional tem dados iniciando antes dos dados
# do Sensor FiledPRO, pois inicia as 00 horas do dia 01/09/20 enquanto o Sensor FieldPRO
# inicia apenas as 23 horas do dia 30/09/20
#Ao mesmo tempo o dataframe Estação Convencional finaliza seus dados antes, pois termina
# as 23 horas do dia 03/12/20, enquanto no Sensor FieldPRO finaliza as 13 horas do dia
# 11/12/20. Por tanto a intersecção de horários é entre 2020-09-30T23:00:00Z e 2020-12-03T23:00:00Z

In [ ]:
#O FOR percorre os dois dataframes procurnando os valores de Datetime que são iguais para criar uma lista
# com os valores de chuva da intersecção
lista_interseccao_chuva = []
for i in range(len(data_estacao_convencional_pronto['Datetime'])):
  for j in range(len(data_sensor_pronto['Datetime'])):
    if data_estacao_convencional_pronto['Datetime'][i] == data_sensor_pronto['Datetime'][j]:
      lista_interseccao_chuva.append(data_estacao_convencional_pronto['chuva'][i])

In [ ]:
#Criando os índices que serão usados para delimitar o dataframe Sensor FiledPRO
# para apenas os datetimes da instersecção
indice_inicio = int(data_sensor_pronto[data_sensor_pronto['Datetime']== data_inicio].index.values)
indice_fim = int(data_sensor_pronto[data_sensor_pronto['Datetime']== data_fim].index.values)+1

In [ ]:
#Criando um novo dataframe para o Senfor FieldPRO com apenas os datetimes da instersecção
data_sensor_intersec = data_sensor_pronto.iloc[indice_inicio:indice_fim]

In [ ]:
data_sensor_intersec

,Datetime,air_humidity_100,air_temperature_100,atm_pressure_main,num_of_resets,piezo_charge,piezo_temperature
0,2020-09-30T23:00:00Z,38.0,31.366,9412,0,45123,30
1,2020-10-01T00:00:00Z,38.5,31.366,9412,0,45025,31
2,2020-10-01T01:00:00Z,39.0,31.366,9419,0,44923,31
3,2020-10-01T02:00:00Z,39.0,31.322,9419,0,44825,31
4,2020-10-01T03:00:00Z,38.0,31.240,9416,0,44728,31
...,...,...,...,...,...,...,...
1524,2020-12-03T19:00:00Z,86.0,20.880,9412,3,35217,20
1525,2020-12-03T20:00:00Z,90.0,20.980,9409,3,35132,19
1526,2020-12-03T21:00:00Z,93.0,20.620,9408,3,35068,20
1527,2020-12-03T22:00:00Z,94.0,20.080,9406,3,35007,19


In [ ]:
#Adicionando a lista de chuva ao dataframe Senfor FieldPRO
data_sensor_intersec['chuva'] = lista_interseccao_chuva
data_sensor_intersec

,Datetime,air_humidity_100,air_temperature_100,atm_pressure_main,num_of_resets,piezo_charge,piezo_temperature,chuva
0,2020-09-30T23:00:00Z,38.0,31.366,9412,0,45123,30,0.0
1,2020-10-01T00:00:00Z,38.5,31.366,9412,0,45025,31,0.0
2,2020-10-01T01:00:00Z,39.0,31.366,9419,0,44923,31,0.0
3,2020-10-01T02:00:00Z,39.0,31.322,9419,0,44825,31,0.0
4,2020-10-01T03:00:00Z,38.0,31.240,9416,0,44728,31,0.0
...,...,...,...,...,...,...,...,...
1524,2020-12-03T19:00:00Z,86.0,20.880,9412,3,35217,20,0.0
1525,2020-12-03T20:00:00Z,90.0,20.980,9409,3,35132,19,0.0
1526,2020-12-03T21:00:00Z,93.0,20.620,9408,3,35068,20,0.0
1527,2020-12-03T22:00:00Z,94.0,20.080,9406,3,35007,19,0.0


In [ ]:
#Criando o dataframe que será usado nos modelos pedritivos com dados do dataframe Sensor FieldPRO exceto pela
# coluna datetime
data_analises = data_sensor_intersec[['air_humidity_100',	'air_temperature_100',
                                      'atm_pressure_main',	'num_of_resets',	'piezo_charge',	'piezo_temperature']]

data_target = data_sensor_intersec[['chuva']]

data_analises_completo = data_sensor_intersec[['air_humidity_100',	'air_temperature_100',
                                      'atm_pressure_main',	'num_of_resets',	'piezo_charge',	'piezo_temperature', 'chuva']]

In [ ]:
parte = round(len(data_analises_completo)*0.8)
treino = data_analises_completo.iloc[0:parte]
teste = data_analises_completo.iloc[parte:len(data_analises_completo)]

In [ ]:

# Dados que queremos prever: PIB, chamaremos de Y
Y = treino[['chuva']]
# Dados que queremos prever: PIB, chamaremos de Y
X = treino[['air_humidity_100',	'air_temperature_100',
                                      'atm_pressure_main',	'num_of_resets',	'piezo_charge',	'piezo_temperature']]

X_prev = teste[['air_humidity_100',	'air_temperature_100',
                                      'atm_pressure_main',	'num_of_resets',	'piezo_charge',	'piezo_temperature']]

In [ ]:
mod1 = LinearRegression()
mod1.fit(X, Y)

LinearRegression()

In [ ]:
Y_prev = mod1.predict(X_prev)
Y_real = teste[['chuva']].to_numpy()

In [ ]:
erro = Y_real - Y_prev
erro_abs_medio = np.mean(abs(erro))
print(f'Erro absoluto médio', erro_abs_medio)
erro_medio_perc = erro_abs_medio/np.mean(Y_real)
print(f'Erro médio percentual', erro_medio_perc)

Erro absoluto médio 0.34009096486930995
Erro médio percentual 2.3544759106336843
